In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import base64
from pymongo import MongoClient


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animal_shelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


###########################################################3
# Query parameters for the different preferred Dog Breeds
# One method (read_preferred_dogs) accept a dictionary of otpion and return object that match those preferences
# Instead of writing different method for each filtering preferences, this one function helps us 
# do more with less in a scalable way. The query dictionaries below are passed to the method for the different filtering operations
dogs_query = {"animal_type":"Dog"}

preferred_breed = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler", "Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland", "Alaskan Malamute", "Old English" 
"Sheepdog", "Siberian Husky"]

disaster_query = {"animal_type":"Dog", "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks":{"$gte":20}, "age_upon_outcome_in_weeks":{"$lte":300}}
water_query = {"animal_type":"Dog", "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome": "Intact Female", "age_upon_outcome_in_weeks":{"$gte":26}, "age_upon_outcome_in_weeks":{"$lte":156}}
mountain_query = {"animal_type":"Dog", "breed":{"$in":["German Shepherd","Alaskan Malamute", "Old English" 
"Sheepdog", "Siberian Husky"]}, "sex_upon_outcome": "Intact Male", "age_upon_outcome_in_weeks":{"$gte":26}, "age_upon_outcome_in_weeks":{"$lte":156}}
###################################################



# class read_all method returns a list of json objects converted from a cursor object. This was done to cast ObjectId to string since it caused problems rendering the table. See read_all method for details
dogs = shelter.read_all(dogs_query) # Query for only animals that are Dogs 

# turn animal json objects to pandas Dataframe object
df = pd.DataFrame(list(dogs))



#########################
# Dashboard Layout / View
#########################
# app = JupyterDash('SimpleExample')
app = dash.Dash(__name__)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    # html.Div(id='hidden-div', style={'display':'none'}),
    html.Div(children = [ 
		 html.Center(children=[
	
	
	html.A(href="https://www.snhu.edu/",
	children=[ html.Img(alt="Link to snhu website",
 src='data:image/png;base64,{}'.format(encoded_image.decode()), 
 style={'width':'70px', 'height':'70px'}, )]
),                              
     html.Span(children= 'Sinde Tabita CS-340 Project Two Dashboard', style={'font-size':'30px', 'font-weight':"bold"}) ]),
]),
    html.Hr(),
    html.Div(
        
# dcc.RadioItems(['Water Rescue', 'Mountain Rescue','Disaster Rescue', "Reset"],  inline=True),

[dcc.RadioItems(
	id='select_radio',
	options=[
			{'label': 'Water Rescue', 'value': 'water'},
			{'label': 'Mountain Rescue', 'value': 'mountain'},
			{'label': 'Disaster Rescue', 'value': 'disaster'},
			{'label': 'Reset', 'value': 'reset'},
	],
	value='',
	labelStyle={'display': 'inline-block'}
    )]),



    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size= 10, 
        style_table={'overflowX': 'auto'},
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[],
        page_action="native",
        page_current= 0,

    ),
    html.Br(),
     html.Hr(),
     
		 html.Div(children=[
	    
	  dcc.Graph(id="graph", style={'width': '49%', 'display': 'inline-block'}
),
     html.Div(
            id='map-id',
            className='col s12 m6',
	    				style={'width': '49%', 'display': 'inline-block'}

            )

		 ] ),
])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_rows):
    print(selected_rows)
    return [{
        'if': { 'row_index': i },
        'background_color': '#D2F3FF'
    } for i in selected_rows]



# Update table with with data obtained with user input
@app.callback(
    Output('datatable-id', 'data'),
    [Input('select_radio', 'value')]
)
def update_table(value):
		if value == "" or value == "reset":
			return df.to_dict('records')
		preferred_dogs = []	
		if value == "disaster":
			preferred_dogs =shelter.read_preferred_dogs(disaster_query)
		if value == "water":
			preferred_dogs =shelter.read_preferred_dogs(water_query)
		if value == "mountain":
			preferred_dogs =shelter.read_preferred_dogs(mountain_query)
			print(value)
		c = pd.DataFrame(list(preferred_dogs))
		return c.to_dict('records')


# Call back to update the pie chart
@app.callback(
Output("graph", "figure"), 
Input('select_radio', 'value'))
def generate_chart(value):
		

		title=""
		pg = []
		if value == "reset" or value == "":
			title = 'Unfiltered Grazioso Salvare Data'
			pg = shelter.read_preferred_dogs(dogs_query)
			pg = [dog for dog in dogs if dog["breed"] in preferred_breed]
		if value == "disaster":
			title = "Dogs Suitable for Disaster Rescue"
			pg =shelter.read_preferred_dogs(disaster_query)
		if value == "water":
			title= "Dogs Suitable for Water Rescue"
			pg =shelter.read_preferred_dogs(water_query)
		if value == "mountain":
			title = "Dogs Suitable for Water Mountain"
			pg =shelter.read_preferred_dogs(mountain_query)
		breed_count = {}
		for dog in pg:
			if not breed_count.get(dog['breed']):
				breed_count[dog['breed']]= 1
			else:
				breed_count[dog['breed']] = breed_count[dog['breed']]+1

		names = list(breed_count.keys())
		values =list(breed_count.values())
		print(names)
		print(values)
		
		
		fig = px.pie(pd.DataFrame(list(pg)), values=values,  names=names,   title =title)
		return fig




# callback handles changes to the maps as a result of the user selecting a row on the table. the animal object that corresponds to the selected row data is used to build the map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'selected_rows')])
def update_map(selected_rows): # update_map receives a list (format [0])  of the id of the selected row		
	if(len(selected_rows))>0: # selected_rows is also an empty list when the table first loads, and no row has been selected. Check for empty list to avoid error in rending map
		selected = dogs[selected_rows[0]]
		print(selected)
		return [
				dl.Map(style={'width': '100vw', 'height': '500px'}, center=[selected["location_lat"],selected["location_long"]], zoom=10, children=[
						dl.TileLayer(id="base-layer-id"),
						# Marker with tool tip and popup
						dl.Marker(position=[selected["location_lat"],selected["location_long"]], children=[
								dl.Tooltip(selected["animal_type"]),
								dl.Popup([
										# Check if name exist to avoid KeyError
										html.H1(selected['name'] if selected.get('name')else ""),
										html.P(f'Date of Birth: {selected["date_of_birth"]}'),
										html.P(f'Age Upon Outcome: {selected["age_upon_outcome"]}'),
										html.P(f'Color: {selected["color"]}'),
										html.P(f'Breed: {selected["breed"]}')
								])
						])
				])
		]
	else:
			return dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48])
        ])
    

    
app

IncorrectTypeException: The input argument `select_radio.value` must be a list or tuple of `dash.dependencies.Input`s.